# Particle Filters

In [2]:
class Map:
    def __init__(self, world_size, landmarks):
        self.world_size = world_size
        self.landmarks = landmarks

In [3]:
from math import *
import random

class Robot:
    def __init__(self, world_size):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0
        self.turn_noise = 0.0
        self.sense_noise = 0.0
        
    def set(self, new_x, new_y, new_orientation, world_size):
        if new_x < 0 or new_x >= world_size:
            raise (ValueError, 'X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise (ValueError, 'Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise (ValueError, 'Orientation must be in [0..2pi]')
        
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
        
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        self.forward_noise = float(new_f_noise)
        self.turn_noise = float(new_t_noise)
        self.sense_noise = float(new_s_noise)
        
    def sense(self, landmarks):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    def move(self, turn, forward, world_size):
        if forward < 0:
            raise (ValueError, 'Robot cant move backwards')
            
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size
        y %= world_size
        self.set(x, y, orientation, world_size)
    
    def Gaussian(self, mu, sigma, x):
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))    
    
    def measurement_prob(self, measurement, landmarks):
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))

In [14]:
import random

class ParticleFilter:
    def __init__(self, gmap, num_particle):
        self.gmap = gmap
        self.N = num_particle
        
    def initialize_particles(self, new_f_noise, new_t_noise, new_s_noise):
        particles = []
        for i in range(self.N):
            p = Robot(self.gmap.world_size)
            p.set_noise(new_f_noise, new_t_noise, new_s_noise)
            particles.append(p)
        return particles
        
    def move_praticles(self, particles, turn, forward):
        moved_p = []
        for p in particles:
            p.move(turn, forward, self.gmap.world_size)
            moved_p.append(p)
        return moved_p
        
    def get_importance_weights(self, measurement, particles):
        weights = []
        for p in particles:
            weights.append(p.measurement_prob(measurement, gmap.landmarks))
            
        sigma = sum(weights)
        print(sigma)
        return weights
    
    def resampe_particles(self, particles, weights):
        resampled_p = []
        idx = int(random.random() * self.N)
        beta = 0.0
        w_max = max(weights)
        
        for i in range(self.N):
            beta += random.random() * 2.0 * w_max
            while weights[idx] < beta:
                beta -= weights[idx]
                idx = (idx + 1) % self.N
            resampled_p.append(particles[idx])
        return resampled_p

In [15]:
def eval(r, p):
    sum = 0.0
    for i in range(len(p)):
        dx = (p[i].x - r.x + world_size/2.0) % world_size - (world_size / 2.0)
        dy = (p[i].y - r.y + world_size/2.0) % world_size - (world_size / 2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))

In [16]:
landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0
gmap = Map(world_size, landmarks)

In [18]:
robot = Robot(gmap.world_size)
robot.move(0.1, 5.0, gmap.world_size)
Z = robot.sense(gmap.landmarks)
#print("Actual distance(between Robot and Landmarks):\n", Z, "\n")
#print("Robot position:\n", robot, "\n")

pf = ParticleFilter(gmap, 1000)
particles = pf.initialize_particles(0.05, 0.05, 5.0)

T = 10
for t in range(T):
    robot.move(0.1, 5.0, gmap.world_size)
    Z = robot.sense(gmap.landmarks)
    #print("Actual distance(between Robot and Landmarks):\n", Z, "\n")
    #print("Robot position:\n", robot, "\n")
    particles = pf.move_praticles(particles, 0.1, 5.0)
    weights = pf.get_importance_weights(Z, particles)
    particles = pf.resampe_particles(particles, weights)
    #for i in range(1000):
    #    print(particles[i], " - ", weights[i])
    print("+++++++++++++++++++")
    
#print("Position of Particles which represent the prediction of Robot position:")
#for i in range(10):
#    print(particles[i])

0.000420794488933016
+++++++++++++++++++
0.00010123913320973617
+++++++++++++++++++
5.672407934083237e-12
+++++++++++++++++++
3.8890826962864475e-32
+++++++++++++++++++
5.2770540238451134e-14
+++++++++++++++++++
4.94140531230577e-13
+++++++++++++++++++
3.847532062350424e-49
+++++++++++++++++++
2.0431723085684807e-40
+++++++++++++++++++
3.5482812169434416e-64
+++++++++++++++++++
2.2401252629722485e-40
+++++++++++++++++++
